In [ ]:
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from keras.applications import vgg19
from keras import backend as K

In [ ]:
# 変換したい画像へのパス
target_image_path = 'test.jpg'

# スタイル画像へのパス
style_reference_image_path = 'images.jpg'

# 生成する画像のサイズ
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [ ]:
def preprocess_img(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_img(x):
    # ImageNetから平均ピクセル値を取り除くことにより、中心を0に設定
    # これにより、vgg19.preprocess_inputによって実行される変換が逆になる
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
    # 画像を'BGR'から'RGB'に変換
    # これもvgg19.preprocess_inputの変換を逆にするための措置
    x = x[:, :, ::-1]
    
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
target_image = K.constant(preprocess_img(target_image_path))
style_reference_image = K.constant(preprocess_img(style_reference_image_path))

# 生成された画像を保持するプレースホルダ
combination_img = K.placeholder((1,img_height, img_width, 3))

# 3つの画像を1つのバッチにまとめる
input_tensor = K.concatenate([target_image, style_reference_image, combination_img], axis=0)

# 3つの画像から成るバッチを入力として使用するVGG19モデルを構築
# このモデルには、学習済みのImageNetの重みが読み込まれる
model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
print('Model loaded.')

In [ ]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

def total_variation_loss(x):
    a = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [ ]:
# 層の名前を活性化テンソルにマッピングするディクショナリ
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# コンテンツの損失関数に使用する層の名前
content_layer = 'block5_conv2'

# スタイルの損失関数に使用する層の名前
style_layers = ['block1_conv1',
               'block2_conv1',
               'block3_conv1',
               'block4_conv1',
               'block5_conv1']

# 損失関数の加重平均の重み
total_varidation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# すべてのコンポーネントをこのスカラー変数に追加することで、損失関数を定義
loss = K.variable(0.)

layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features, combination_features)

# 各ターゲット層のスタイルの損失関数を追加
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
    
loss += total_varidation_weight * total_variation_loss(combination_img)

In [ ]:
# 損失関数をもとに、生成された画像の勾配を取得
grads = K.gradients(loss, combination_img)[0]

# 現在の損失関数の値と勾配の値を取得する関数
fetch_loss_and_grads = K.function([combination_img], [loss, grads])

# このクラスは、損失関数の値と勾配の値を２つのメソッド呼び出しを通じて取得できるようにfetch_loss_and_gradsをラッピングする。
# この２つのメソッド呼び出しは、ここで使用するScipyのオプティマイザによって要求される
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_value = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
    
evaluater = Evaluator()

In [ ]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = 'style_transfer_result'
iterations = 20

# 初期状態：ターゲット画像
x = preprocess_img(target_image_path)

# 画像を平坦化：scipy.optimize.fmin_l_bfgs_bは一次元ベクトルしか処理しない
x = x.flatten()

for i in range (iterations):
    print('Start of iteration', i)
    start_time = time.time()
    # ニューラルスタイル変換の損失関数を最小化するために、生成された画像のピクセルにわたってL-BFGS最適化を実行
    # 損失関数を計算する関数と勾配を計算する関数を2つの別々の引数として渡さなければならないことに注意
    x, min_val, info = fmin_l_bfgs_b(evaluater.loss, x, fprime=evaluater.grads, maxfun=20)
    print('Current loss value:', min_val)
    # この時点で生成された画像を保存
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_img(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))